## ------------------ Module 2: Simulate Algo ------------------

- What: Run algo over dates in dataset
- When: 25 May 2022  

- Weighted cost removed from buy factor (17 May)
- Max maturity variable
- Max downturn (now using low price)
- Functions added to use Buy/Sell using high/low

#### Comments
1. Sell action takes priority over a Buy action
2. Multiple actions are permitted at a timestamp (i.e. can progress through levels if conditions met)
3. Algo runs till end of dataset if max level reached (max running time now available).  Sell on last date.  Last dates are artifical so can be identified.
4. Can load config from file or define as required.  Not guaranteed to hit capital (i.e. 72) but can define separately.

$\color{red}{\text{Define Parameters:}}$

In [2]:
# Location of resampled data
FileName   = ''

# Location of config file (if used)
ConfigFile = ''

# Location to push results to.
SaveFile   = ''

#### 1. Load some libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

# Little function to keep track of code execution.
def Keep_Track():
    print("Executed successfully. " + dt.datetime.now().strftime("%a %d %b @ %H:%M:%S") + "\n")

Keep_Track()

Executed successfully. Wed 25 May @ 12:51:05



#### 2. Load some data.

2.1 Create a function to load the dataset and perform some simple calculations

In [4]:
# Function to load the data

# Input    : File name - for example 'kline_1m_apebusd.csv'
# Output   : a timestamp order dataset with two records added in the future to ensure algo terminates with an action.
# LastTrade: The final trade date (we add artifical ones so these need to be removed later.) 

def Load_Data(File_Name):
    
    print('* Loading', File_Name)
    
    try:
    
        # Load the transaction dataset.
        Data = pd.read_csv(File_Name)

        # Convert the time to a timestamp - need to specify the format is day first to avoid parsing errors.
        Data['start_timestamp'] = pd.to_datetime(Data['start_timestamp'], dayfirst=True)

        # Remove some odd values from the start - not necessary but they are probably not representative.
        print('* Filtering ...')
        Data = Data[Data['open_price']>1.0001]

        # Provide a quick summary of how many records we have
        print('* Data loaded   :', len(Data),'records.')
        print('* First Date    :', np.min(Data['start_timestamp']))
        print('* Last trade    :', np.max(Data['start_timestamp']))
        
        LastTrade = np.max(Data['start_timestamp'])

        # Add in two trades at the end of the dataset to ensure we always get at least
        # one buy and one sell.  Later we use argmax to find the maximal position of an event,
        # if no event is found this will return 0 which is not what we want.

        End_Sell_Record = Data[Data['start_timestamp']==np.max(Data['start_timestamp'])].copy()
        # We will always choose to sell with such a high close
        End_Sell_Record['close_price'] = 10000
        End_Sell_Record['start_timestamp'] = End_Sell_Record['start_timestamp'] + pd.offsets.DateOffset(years=10)
        Data = pd.concat([Data,End_Sell_Record])

        print('* Last Sell Date:', np.max(Data['start_timestamp']), '--added to ensure at least one sell date.')

        End_Buy_Record = Data[Data['start_timestamp']==np.max(Data['start_timestamp'])].copy()
        # We will always choose to buy with such a low close (even with config set to not buy)
        End_Buy_Record['close_price'] = 0.00001
        End_Buy_Record['start_timestamp'] = End_Buy_Record['start_timestamp'] + pd.offsets.DateOffset(years=20)
        Data = pd.concat([Data,End_Buy_Record])

        print('* Last Buy Date :', np.max(Data['start_timestamp']), '--added to ensure at least one buy date.')

        # Order the data by time and reset the index of the dataframe.
        Data = Data.sort_values(by=['start_timestamp']).reset_index()
        
        print('* ---------- Load complete.')
        
        return Data, LastTrade
    
    except:
        print('* ---------- Load FAILED!')


print('* Load function created.')
print('')
Keep_Track()

* Load function created.

Executed successfully. Wed 25 May @ 12:51:05



2.2 Create a function to create some Algo configuration

In [5]:
# Function to create an algo configuration

# TopUp_Step: Define the number of steps to repeat a top-up factor
# TopUp_Delta: Define the change in Top up when the step changes
# TopUp_Percentage: Define the top-up % - this controls the reduction in the price for each buy
# Total_Capital: Define the total capital level

# Returns a dataframe so as to be consistent with loading a csv.

# Define the function which does the job
def Build_Config(Total_Capital = 72, TopUp_Step = 2, TopUp_Delta = 0.2,TopUp_Percentage = 0.005):
    
    # Calculate the capital schedule.
    Capital_Factor = np.repeat(np.arange(start = 1, step = TopUp_Delta, stop = Total_Capital),TopUp_Step)
    Cumulative_Sum = np.cumsum(Capital_Factor)
    Capital_Factor = Capital_Factor[Cumulative_Sum <= Total_Capital]

    # Calculate the price float
    PriceFloat = np.arange(start = 1, step = -TopUp_Percentage, stop = 0)
    PriceFloat  = PriceFloat[:len(Capital_Factor)]

    # Calculate the capital consumption (as a percentage)
    Capital_Consumption = 100*np.cumsum(Capital_Factor) / np.sum(Capital_Factor)

    # Calculate the pricefloat_h0
    pricefloat_h0 = np.cumsum(PriceFloat * Capital_Factor) / np.cumsum(Capital_Factor)
    
    # Add in total captial to be clear.
    Capital = np.repeat(np.sum(Capital_Factor), len(Capital_Factor))
    
    # Add in a level indicator
    Levels = ['Level ' + str(x) for x in np.arange(0,len(Capital_Factor))]
    
    Pre_Result = pd.DataFrame({'Level': Levels, 'TopUp Factor':Capital_Factor, 'Capital Consumption':Capital_Consumption,
                 'pricefloat_0':PriceFloat, 'pricefloat_h0':pricefloat_h0, 'TotalCap':Capital})
    
    # Return the result
    return Pre_Result

print('* Build config function complete.')
print('')
Keep_Track()

* Build config function complete.

Executed successfully. Wed 25 May @ 12:51:05



2.3 Create a function to either load the config from file or use the function

In [6]:
# Load the config file.
# Can load either from a file or from the config builder.

# Load_file: Load from a file or not.
# File_Name: The name of the file.  For example '029_2_005_006.csv'
# Remaining are defined in the previous function definition.

def Load_Config(Load_file, File_Name = 'none', Total_Capital = 72, TopUp_Step = 2, TopUp_Delta = 0.2,TopUp_Percentage = 0.005):
    
    try:
        if Load_file:

            print('* Loading file:', File_Name)
            # Load the config file that determines the algo
            Config = pd.read_csv(File_Name)

            # Parse the capital as header has %
            Capital = Config['Capital Consumption %'].str.rstrip('%').astype('float')

            Buy_Rule = Config['pricefloat_0']
            Weighted_Cost = Config['pricefloat_h0']

        else:
            print('* Running config builder:', Total_Capital,'|',TopUp_Step,'|',TopUp_Delta,'|',TopUp_Percentage)
            Config = Build_Config(Total_Capital, TopUp_Step, TopUp_Delta,TopUp_Percentage) 

            # Load the Buy and weighted cost rules for the calculation
            Buy_Rule = Config['pricefloat_0']
            Weighted_Cost = Config['pricefloat_h0']
            Capital = Config['Capital Consumption']

        # We need to add a final level which dictates what we do if we reach the last level.  Since there
        # are no more buy or sells.  In this case we hold until the end of the data set (which forces a sell) or
        # we sell once the profit return is some level.
        Weighted_Cost = np.append(Weighted_Cost, Weighted_Cost[Capital == 100])

        # Capital is unchanged
        Capital = np.append(Capital, 100.00)

        # We do not make any further purchases.
        Buy_Rule = np.append(Buy_Rule, 0.001)


        print('* Config loaded:', len(Config),'records.')
        print('* ---------- Load complete.')
        
        return Buy_Rule, Weighted_Cost, Capital, Config
    
    except:  
        print('* Error with config creation.')
        print('* ---------- Config FAILED!')

print('* Build config function created.')
print('')
Keep_Track()

* Build config function created.

Executed successfully. Wed 25 May @ 12:51:05



#### 3. Calculate Functions

3.1 Create a function which run the algo over each starting date in the dataset.

In [7]:
# Code to run algo over entire dataset

# Trading_Data     : Data to run the calculation for
# Profit_Threshold : Threshold (e.g. 1.006)
# DataSet          : The trade dataset with the trades and the interesting date
# Buy_Config       : Buy config (0.995, 0.99 etc) 
# Weighted_Cost_Config  : Weighted cost
# Capital_Config   : Capital allocation percentage.
# Drop             : The max drop from initial price before sell action is automatically triggered (0.5 = 50% fall).
# Max_Mat          : The max length of a single algo run before a sell action is automatically triggered (in minutes).
# Steps            : Number of steps to split calculation into (handle memory issues) 

def Run(Trading_Data, Buy_Config, Weighted_Cost_Config, Capital_Config, Drop = 0.5, Max_Mat = 120, Steps = 10, Profit_Threshold = 1.006):

    print("Start time:" + dt.datetime.now().strftime("%a %d %b @ %H:%M:%S") + "\n")
    
    # The price fall is input as a %, say 10%.  This means we need to consider Close/Initial <= 0.9.
    Drop = 1 - Drop
    
    # Define some key arrays of data.
    Dates = np.array(Trading_Data['start_timestamp'])

    # All opening prices in the dataset
    Open = np.array(Trading_Data['open_price'])

    # All closing prices in the dataset
    Close = np.array(Trading_Data['close_price'])

    print('* Open and close arrays constructed')
    
    # Constant to hold how many dates we have to start the algo from.
    Data_Length  = len(Trading_Data)

    # Constant to hold how many levels we need to consider
    Level_Length  = len(Buy_Config)

    # How many steps to run the calculation - needed for outer join as large amount of memory is needed.
    Breakdown = np.linspace(start = 0, stop = Data_Length, num = Steps, endpoint = True, dtype = int)

    # Arrays to store the results.  These will contain the index of the date for each type of action i.e.
    # buy or sell for each start date at each level.
    Buy_Summary = np.zeros((Level_Length-1,Data_Length))
    Sell_Summary = np.zeros((Level_Length-1,Data_Length))

    print('* Profit threshold            :', Profit_Threshold)
    print('* Total Levels to consider    :', Level_Length)
    print('* Total start dates in dataset:', Data_Length)
    print('* Steps set                   :', Steps)
    print('* Groups                      :', Breakdown)

    # Loop over all but the last so we can hit the last in the last iteration
    for ind, StartIndex in enumerate(Breakdown[:-1]):

        # Calculate the step - this is how many start dates we consider in this loop
        Adjust = Breakdown[ind+1] - StartIndex   
        print('')
        print('* Start:', StartIndex)
        print('   -Level: 1|',end='')

        # Collect the opening prices we will consider in this loop.
        New_Open = Open[StartIndex:StartIndex + Adjust]

        # Collect all closing prices from startindex to the end of the data set.
        New_Close = Close[StartIndex:]

        # Calculate the lower triangular matrix of the outer join.  This computes the profit for each 
        # possible open price at each subsequent closing price.  The lower triangular matrix is needed
        # since we only consider actions/profit after the opening date.
        Profit = np.tril(np.outer(New_Close,1/New_Open))
        
        # Calculate an early termination mask which enforces a sell if algo has been running for Max_Mat mins
        Max_Mat_Mask = np.tril(Profit, k = -Max_Mat) > 0

        # Calculate the first time the profit exceeds some level (this is when we sell)
        # We need to add StartIndex since the index of the array resets to zero for each loop so
        # is needed once the array is concatenated.
#         Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Profit >= Profit_Threshold, axis = 0) + StartIndex
#         Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax((Profit >= Profit_Threshold) | (Profit <= Drop), axis = 0) + StartIndex
        Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | ((Profit >= Profit_Threshold) | ((Profit <= Drop) & (Profit > 0))), axis = 0) + StartIndex

        
        # Calculate a mask - this keeps track of the earliest the next buy or sell is permitted.  For example if we
        # have a buy action then the next level will only allow actions on or AFTER this date.
        # Buy_Rule[1] is the condition to buy to enter Level 1 (0.995)
        Buy_Mask = np.cumsum((Profit <= Buy_Config[1]) & (Profit>0) , axis = 0) > 0 

        # Store the results.
        Buy_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex

        # Now loop over the remaining levels ...
        for idx, x in enumerate(Buy_Config[2:]):
            # Note here idx will be zero at the start of the loop, so we need to add 2 since we start at L2
            print('{}|'.format(idx+2), end='')

            # Compute the thresholds to buy or sell using the config data.
            Sell_Factor = Weighted_Cost_Config[idx+1] * Profit_Threshold
            
#             Buy_Factor  = Weighted_Cost_Config[idx+1] * x
            Buy_Factor  = x

            # Store the results (note: Buy_Mask keeps track of the last buy action)
#             Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax((Profit>=Sell_Factor) & Buy_Mask, axis = 0) + StartIndex
#             Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(((Profit>=Sell_Factor) | (Profit <= Drop)) & Buy_Mask, axis = 0) + StartIndex
            Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | (((Profit>=Sell_Factor) | ((Profit <= Drop) & (Profit > 0))) & Buy_Mask), axis = 0) + StartIndex

            # Need to keep track of this for the next loop
            Buy_Mask = np.cumsum((Profit<=Buy_Factor) & Buy_Mask , axis = 0) > 0
    #         Buy_Mask = np.cumsum(np.cumsum((Profit<=Buy_Factor) & Buy_Mask , axis = 0), axis = 0) > 1

            # Keep the results.
            Buy_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex

    print('')
    print('* Analysis complete.')
    print('* Creating summary ...')
    
    # First_Sell_Index: The index of the first sell action
    # Capital_Level   : The capital utilisation at the first sell action
    # Final_Price     : The closing price at the first sell action
    # Cost            : The weighted cost at the first sell action (this is level depended)
    First_Sell_Index, Capital_Level, Final_Price, Cost = np.empty(0), np.empty(0), np.empty(0), np.empty(0)

    # The date at the first sell action.
    End_Date = np.empty(0, dtype ='datetime64')

    # Find the first time the Sell date is lower than the buy date.  At this time the algo ends.
    # Need to determine what we do if we have equality ... which takes priority - here sell does.
    First_Sell_Level = np.argmax(np.cumsum(Buy_Summary >= Sell_Summary, axis=0) > 0, axis = 0)+1

    # Calculate the index of the date for the first sell.
    for idx, i in enumerate(First_Sell_Level):
        First_Sell_Index = np.append(First_Sell_Index, Sell_Summary[i-1][idx])
        Capital_Level = np.append(Capital_Level, Capital_Config[i-1])
        Final_Price = np.append(Final_Price, Close[int(Sell_Summary[i-1][idx])])
        Cost = np.append(Cost, Weighted_Cost_Config[i-1])
        End_Date = np.append(End_Date, Dates[int(Sell_Summary[i-1][idx])])

    # Collect the results in a dataframe    
    Results = pd.DataFrame({'Start_Date':Data['start_timestamp'][:Data_Length], 
                            'End_Date':End_Date,
                            'Initial_Open':Open[:Data_Length],
                            'Final_Close':Final_Price,                        
                            'First_Sell':First_Sell_Index, 'Level':First_Sell_Level-1,
                            'Capital' :Capital_Level,
                            'Cost':Cost})


    # Since we don't know the number of levels the level fields are calculated in a loop.
    for levels in range(Level_Length-1):
        Results['L'+str(levels+1)+'_Buy'] = Buy_Summary[int(levels)]
        Results['L'+str(levels+1)+'_Sell'] = Sell_Summary[int(levels)]

    Results['Life(s)'] = (Results['End_Date'] - Results['Start_Date']).astype('timedelta64[s]')

    # Calculate some Capital return.
    Results['Cap_Return%'] = Results['Capital'] * (Results['Final_Close'] / (Results['Initial_Open']*Results['Cost']) - 1)

    Results['Threshold'] = Profit_Threshold
    Results['Max_Drop'] = Drop
    Results['Max_Mat'] = Max_Mat
    
    Results['Sell_Price'] = round(Results['Initial_Open']*Results['Cost']*Profit_Threshold,2)
    
    print('* Complete')
    
    return Results

print('* Run all function Constructed.')

print('')
Keep_Track()

* Run all function Constructed.

Executed successfully. Wed 25 May @ 12:51:05



In [8]:
# Code to run algo over entire dataset - this  version uses the high and low prices over the interval 
# to determine buy and sell actions (to match Tina approach 25 May)

# Trading_Data     : Data to run the calculation for
# Profit_Threshold : Threshold (e.g. 1.006)
# DataSet          : The trade dataset with the trades and the interesting date
# Buy_Config       : Buy config (0.995, 0.99 etc) 
# Weighted_Cost_Config  : Weighted cost
# Capital_Config   : Capital allocation percentage.
# Drop             : The max drop from initial price before sell action is automatically triggered (0.5 = 50% fall).
# Max_Mat          : The max length of a single algo run before a sell action is automatically triggered (in minutes).
# Steps            : Number of steps to split calculation into (handle memory issues) 

def Run_HL(Trading_Data, Buy_Config, Weighted_Cost_Config, Capital_Config, Drop = 0.5, Max_Mat = 120, Steps = 10, Profit_Threshold = 1.006):

    print("Start time:" + dt.datetime.now().strftime("%a %d %b @ %H:%M:%S") + "\n")
    
    # The price fall is input as a %, say 10%.  This means we need to consider Close/Initial <= 0.9.
    Drop = 1 - Drop
    
    # Define some key arrays of data.
    Dates = np.array(Trading_Data['start_timestamp'])

    # All opening prices in the dataset
    Open = np.array(Trading_Data['open_price'])

    # All closing prices in the dataset
    Low = np.array(Trading_Data['low_price'])

    # All closing prices in the dataset
    High = np.array(Trading_Data['high_price'])    
    
    print('* Open, high and low arrays constructed')
    
    # Constant to hold how many dates we have to start the algo from.
    Data_Length  = len(Trading_Data)

    # Constant to hold how many levels we need to consider
    Level_Length  = len(Buy_Config)

    # How many steps to run the calculation - needed for outer join as large amount of memory is needed.
    Breakdown = np.linspace(start = 0, stop = Data_Length, num = Steps, endpoint = True, dtype = int)

    # Arrays to store the results.  These will contain the index of the date for each type of action i.e.
    # buy or sell for each start date at each level.
    Buy_Summary = np.zeros((Level_Length-1,Data_Length))
    Sell_Summary = np.zeros((Level_Length-1,Data_Length))

    print('* Profit threshold            :', Profit_Threshold)
    print('* Total Levels to consider    :', Level_Length)
    print('* Total start dates in dataset:', Data_Length)
    print('* Steps set                   :', Steps)
    print('* Groups                      :', Breakdown)

    # Loop over all but the last so we can hit the last in the last iteration
    for ind, StartIndex in enumerate(Breakdown[:-1]):

        # Calculate the step - this is how many start dates we consider in this loop
        Adjust = Breakdown[ind+1] - StartIndex   
        print('')
        print('* Start:', StartIndex)
        print('   -Level: 1|',end='')

        # Collect the opening prices we will consider in this loop.
        New_Open = Open[StartIndex:StartIndex + Adjust]

        # Collect all low prices from startindex to the end of the data set.
        New_Low = Low[StartIndex:]

        # Collect all low prices from startindex to the end of the data set.
        New_High = High[StartIndex:]        
        
        # Calculate the lower triangular matrix of the outer join.  This computes the profit for each 
        # possible open price at each subsequent closing price.  The lower triangular matrix is needed
        # since we only consider actions/profit after the opening date.
        Profit_Sell = np.tril(np.outer(New_High,1/New_Open))
        Profit_Buy = np.tril(np.outer(New_Low,1/New_Open))
        
        # Calculate an early termination mask which enforces a sell if algo has been running for Max_Mat mins
        Max_Mat_Mask = np.tril(Profit_Sell, k = -Max_Mat) > 0

        # Calculate the first time the profit exceeds some level (this is when we sell)
        # We need to add StartIndex since the index of the array resets to zero for each loop so
        # is needed once the array is concatenated.
#         Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Profit >= Profit_Threshold, axis = 0) + StartIndex
#         Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax((Profit >= Profit_Threshold) | (Profit <= Drop), axis = 0) + StartIndex
        Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | ((Profit_Sell >= Profit_Threshold) | ((Profit_Buy <= Drop) & (Profit_Buy > 0))), axis = 0) + StartIndex

        
        # Calculate a mask - this keeps track of the earliest the next buy or sell is permitted.  For example if we
        # have a buy action then the next level will only allow actions on or AFTER this date.
        # Buy_Rule[1] is the condition to buy to enter Level 1 (0.995)
        Buy_Mask = np.cumsum((Profit_Buy <= Buy_Config[1]) & (Profit_Buy > 0) , axis = 0) > 0 

        # Store the results.
        Buy_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex

        # Now loop over the remaining levels ...
        for idx, x in enumerate(Buy_Config[2:]):
            # Note here idx will be zero at the start of the loop, so we need to add 2 since we start at L2
            print('{}|'.format(idx+2), end='')

            # Compute the thresholds to buy or sell using the config data.
            Sell_Factor = Weighted_Cost_Config[idx+1] * Profit_Threshold
            
#             Buy_Factor  = Weighted_Cost_Config[idx+1] * x
            Buy_Factor  = x

            # Store the results (note: Buy_Mask keeps track of the last buy action)
#             Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax((Profit>=Sell_Factor) & Buy_Mask, axis = 0) + StartIndex
#             Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(((Profit>=Sell_Factor) | (Profit <= Drop)) & Buy_Mask, axis = 0) + StartIndex
            Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | (((Profit_Sell>=Sell_Factor) | ((Profit_Buy <= Drop) & (Profit_Buy > 0))) & Buy_Mask), axis = 0) + StartIndex

            # Need to keep track of this for the next loop
            Buy_Mask = np.cumsum((Profit_Buy <= Buy_Factor) & Buy_Mask , axis = 0) > 0
    #         Buy_Mask = np.cumsum(np.cumsum((Profit<=Buy_Factor) & Buy_Mask , axis = 0), axis = 0) > 1

            # Keep the results.
            Buy_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex

    print('')
    print('* Analysis complete.')
    print('* Creating summary ...')
    
    # First_Sell_Index: The index of the first sell action
    # Capital_Level   : The capital utilisation at the first sell action
    # Final_Price     : The closing price at the first sell action
    # Cost            : The weighted cost at the first sell action (this is level depended)
    First_Sell_Index, Capital_Level, Final_Price, Cost = np.empty(0), np.empty(0), np.empty(0), np.empty(0)

    # The date at the first sell action.
    End_Date = np.empty(0, dtype ='datetime64')

    # Find the first time the Sell date is lower than the buy date.  At this time the algo ends.
    # Need to determine what we do if we have equality ... which takes priority - here sell does.
    First_Sell_Level = np.argmax(np.cumsum(Buy_Summary >= Sell_Summary, axis=0) > 0, axis = 0)+1

    # Calculate the index of the date for the first sell.
    for idx, i in enumerate(First_Sell_Level):
        First_Sell_Index = np.append(First_Sell_Index, Sell_Summary[i-1][idx])
        Capital_Level = np.append(Capital_Level, Capital_Config[i-1])
        Final_Price = np.append(Final_Price, High[int(Sell_Summary[i-1][idx])])
        Cost = np.append(Cost, Weighted_Cost_Config[i-1])
        End_Date = np.append(End_Date, Dates[int(Sell_Summary[i-1][idx])])

    # Collect the results in a dataframe    
    Results = pd.DataFrame({'Start_Date':Data['start_timestamp'][:Data_Length], 
                            'End_Date':End_Date,
                            'Initial_Open':Open[:Data_Length],
                            'Final_Close':Final_Price,                        
                            'First_Sell':First_Sell_Index, 'Level':First_Sell_Level-1,
                            'Capital' :Capital_Level,
                            'Cost':Cost})


    # Since we don't know the number of levels the level fields are calculated in a loop.
    for levels in range(Level_Length-1):
        Results['L'+str(levels+1)+'_Buy'] = Buy_Summary[int(levels)]
        Results['L'+str(levels+1)+'_Sell'] = Sell_Summary[int(levels)]

    Results['Life(s)'] = (Results['End_Date'] - Results['Start_Date']).astype('timedelta64[s]')

    # Calculate some Capital return.
    Results['Cap_Return%'] = Results['Capital'] * (Results['Final_Close'] / (Results['Initial_Open']*Results['Cost']) - 1)

    Results['Threshold'] = Profit_Threshold
    Results['Max_Drop'] = Drop
    Results['Max_Mat'] = Max_Mat
    
    Results['Sell_Price'] = round(Results['Initial_Open']*Results['Cost']*Profit_Threshold,2)
    
    print('* Complete')
    
    return Results

print('* Run all function Constructed.')

print('')
Keep_Track()

* Run all function Constructed.

Executed successfully. Wed 25 May @ 12:51:05



3.2 Create a function to run the algo with a single start date only

In [9]:
# Code for a single date
# Interesting_Date : Date to run the calculation for
# Profit_Threshold : Threshold (e.g. 1.006)
# DataSet          : The trade dataset with the trades and the interesting date
# Buy_Config       : Buy config (0.995, 0.99 etc) 
# Weighted_Config  : Weighted cost
# Capital_Config   : Capital allocation percentage.

def Run_One_Date(Interesting_Date, Profit_Threshold, DataSet, Buy_Config, Weighted_Config, Capital_Config,Drop = 0.5, Max_Mat = 120):

    # Need to capture the fact the start relative to the underlying data is not at zero
#     ReIndex = min(DataSet[DataSet['start_timestamp'] >= Interesting_Date].index)
    
    ReIndex = 0
    
    # The price fall is input as a %, say 10%.  This means we need to consider Close/Initial <= 0.9.
    Drop = 1 - Drop
    
    # Filter the dataset to start at the above date.
    One_Date = DataSet[DataSet['start_timestamp'] >= Interesting_Date].copy()

    # Define some key arrays of data.
    One_Dates = np.array(One_Date['start_timestamp'])

    # All opening prices in the dataset
    One_Open = np.array(One_Date['open_price'])

    # All closing prices in the dataset
    One_Close = np.array(One_Date['close_price'])

#     print('* Open and close arrays constructed')

    # Constant to hold how many dates we have to run the algo over
    Data_Length  = len(One_Date)

    # Constant to hold how many levels we need to consider
    Level_Length  = len(Buy_Rule)

    # Arrays to store the results.  These will contain the index of the date for each type of action i.e.
    # buy or sell for each start date at each level.
    Buy_Summary = np.zeros((Level_Length-1,1))
    Sell_Summary = np.zeros((Level_Length-1,1))

#     print('* Interesting Date            :', Interesting_Date)
#     print('* Profit threshold            :', Profit_Threshold)
#     print('* Total Levels to consider    :', Level_Length)
#     print('* Total start dates in dataset:', Data_Length)
    
    # Calculate the step - this is how many start dates we consider in this loop
    # In this case we just have one data so it is easy.
    Adjust = 1 
    StartIndex = 0
#     print('')
#     print('* Start:')
#     print('   -Level: 1|',end='')

    # Collect the opening prices we will consider in this loop.
    New_Open = One_Open[0:StartIndex + Adjust]
    
    # Collect all closing prices from startindex to the end of the data set.
    New_Close = One_Close[StartIndex:]

    # Calculate the lower triangular matrix of the outer join.  This computes the profit for each 
    # possible open price at each subsequent closing price.  The lower triangular matrix is needed
    # since we only consider actions/profit after the opening date.
    Profit = np.tril(np.outer(New_Close,1/New_Open))
    
    # Calculate an early termination mask which enforces a sell if algo has been running for Max_Mat mins
    Max_Mat_Mask = np.tril(Profit, k = -Max_Mat) > 0    

    # Calculate the first time the profit exceeds some level (this is when we sell)
    # We need to add StartIndex since the index of the array resets to zero for each loop so
    # is needed once the array is concatenated.
    Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | ((Profit >= Profit_Threshold) | ((Profit <= Drop) & (Profit > 0))), axis = 0) + StartIndex + ReIndex

    # Calculate a mask - this keeps track of the earliest the next buy or sell is permitted.  For example if we
    # have a buy action then the next level will only allow actions on or AFTER this date (hence the double cumsum).
    # Buy_Rule[1] is the condition to buy to enter Level 1 (0.995)
    Buy_Mask = np.cumsum((Profit <= Buy_Config[1]) & (Profit>0) , axis = 0) > 0 
    
    # Store the results.
    Buy_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex + ReIndex
    
    # Now loop over the remaining levels ...
    for idx, x in enumerate(Buy_Config[2:]):
        # Note here idx will be zero at the start of the loop, so we need to add 2 since we start at L2
#         print('{}|'.format(idx+2), end='')

        # Compute the thresholds to buy or sell using the config data.
        Sell_Factor = Weighted_Config[idx+1] * Profit_Threshold
#         Buy_Factor  = Weighted_Config[idx+1] * x
        Buy_Factor  = x

        # Store the results (note: Buy_Mask keeps track of the last buy action)
        Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask |(((Profit>=Sell_Factor)| ((Profit <= Drop) & (Profit > 0))) & Buy_Mask), axis = 0) + StartIndex + ReIndex

        # Need to keep track of this for the next loop
        Buy_Mask = np.cumsum((Profit<=Buy_Factor) & Buy_Mask , axis = 0) > 0

        # Keep the results.
        Buy_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex + ReIndex

    # Set up some arrays to store the results for the start dates.

    # First_Sell_Index: The index of the first sell action
    # Capital_Level   : The capital utilisation at the first sell action
    # Final_Price     : The closing price at the first sell action
    # Cost            : The weighted cost at the first sell action (this is level depended)
    First_Sell_Index, Capital_Level, Final_Price, Cost = np.empty(0), np.empty(0), np.empty(0), np.empty(0)

    # The date at the first sell action.
    End_Date = np.empty(0, dtype ='datetime64')

    # Find the first time the Sell date is lower than the buy date.  At this time the algo ends.
    # Need to determine what we do if we have equality ... which takes priority - here sell does.
    First_Sell_Level = np.argmax(np.cumsum(Buy_Summary >= Sell_Summary, axis=0) > 0, axis = 0) + 1

    # Calculate the index of the date for the first sell.
    for idx, i in enumerate(First_Sell_Level):
        First_Sell_Index = np.append(First_Sell_Index, Sell_Summary[i-1][idx])
        Capital_Level = np.append(Capital_Level, Capital_Config[i-1])
        Final_Price = np.append(Final_Price, One_Close[int(Sell_Summary[i-1][idx])])
        Cost = np.append(Cost, Weighted_Config[i-1])
        End_Date = np.append(End_Date, One_Dates[int(Sell_Summary[i-1][idx])])

    # Collect the results in a dataframe    
    One_Results = pd.DataFrame({'Start_Date':One_Dates[0], 
                            'End_Date':End_Date,
                            'Initial_Open':One_Open[0],
                            'Final_Close':Final_Price,                        
                            'First_Sell':First_Sell_Index, 
                            'Level':First_Sell_Level-1,
                            'Capital' :Capital_Level,
                            'Cost':Cost})


    # Since we don't know the number of levels the level fields are calculated in a loop.
    for levels in range(Level_Length-1):
        One_Results['L'+str(levels+1)+'_Buy'] = Buy_Summary[int(levels)]
        One_Results['L'+str(levels+1)+'_Sell'] = Sell_Summary[int(levels)]

    One_Results['Life(s)'] = (One_Results['End_Date'] - One_Results['Start_Date']).astype('timedelta64[s]')

    # Calculate some Capital return.
    One_Results['Cap_Return%'] = One_Results['Capital'] * (One_Results['Final_Close'] / (One_Results['Initial_Open']*One_Results['Cost']) - 1)
    
    One_Results['Threshold'] = Profit_Threshold
    One_Results['Max_Drop'] = Drop
    One_Results['Max_Mat'] = Max_Mat
    
    One_Results['Sell_Price'] = round(One_Results['Initial_Open']*One_Results['Cost']*Profit_Threshold,2)
    
    
    return One_Results

print('* Run one function Constructed.')

print('')
Keep_Track()

* Run one function Constructed.

Executed successfully. Wed 25 May @ 12:51:05



In [10]:
# Code for a single date - this considers to sell on the high price and buy on the low price.
# Interesting_Date : Date to run the calculation for
# Profit_Threshold : Threshold (e.g. 1.006)
# DataSet          : The trade dataset with the trades and the interesting date
# Buy_Config       : Buy config (0.995, 0.99 etc) 
# Weighted_Config  : Weighted cost
# Capital_Config   : Capital allocation percentage.

def Run_One_Date_HL(Interesting_Date, Profit_Threshold, DataSet, Buy_Config, Weighted_Config, Capital_Config,Drop = 0.5, Max_Mat = 120):

    # Need to capture the fact the start relative to the underlying data is not at zero
#     ReIndex = min(DataSet[DataSet['start_timestamp'] >= Interesting_Date].index)
    
    ReIndex = 0
    
    # The price fall is input as a %, say 10%.  This means we need to consider Close/Initial <= 0.9.
    Drop = 1 - Drop
    
    # Filter the dataset to start at the above date.
    One_Date = DataSet[DataSet['start_timestamp'] >= Interesting_Date].copy()

    # Define some key arrays of data.
    One_Dates = np.array(One_Date['start_timestamp'])

    # All opening prices in the dataset
    One_Open = np.array(One_Date['open_price'])

    # All low prices over the period
    One_Low = np.array(One_Date['low_price'])

    # All high prices over the period
    One_High = np.array(One_Date['high_price'])
    
    # Constant to hold how many dates we have to run the algo over
    Data_Length  = len(One_Date)

    # Constant to hold how many levels we need to consider
    Level_Length  = len(Buy_Rule)

    # Arrays to store the results.  These will contain the index of the date for each type of action i.e.
    # buy or sell for each start date at each level.
    Buy_Summary = np.zeros((Level_Length-1,1))
    Sell_Summary = np.zeros((Level_Length-1,1))
    
    # Calculate the step - this is how many start dates we consider in this loop
    # In this case we just have one data so it is easy.
    Adjust = 1 
    StartIndex = 0

    # Collect the opening prices we will consider in this loop.
    New_Open = One_Open[0:StartIndex + Adjust]
    
    # Collect all low prices from startindex to the end of the data set.
    New_One_Low = One_Low[StartIndex:]

    # Collect all high prices from startindex to the end of the data set.
    New_One_High = One_High[StartIndex:]    
    
    # Calculate the lower triangular matrix of the outer join.  This computes the profit for each 
    # possible open price at each subsequent closing price.  The lower triangular matrix is needed
    # since we only consider actions/profit after the opening date.
    Profit_Sell = np.tril(np.outer(New_One_High,1/New_Open))
    Profit_Buy = np.tril(np.outer(New_One_Low,1/New_Open))
    
    # Calculate an early termination mask which enforces a sell if algo has been running for Max_Mat mins
    Max_Mat_Mask = np.tril(Profit_Sell, k = -Max_Mat) > 0    

    # Calculate the first time the profit exceeds some level (this is when we sell)
    # We need to add StartIndex since the index of the array resets to zero for each loop so
    # is needed once the array is concatenated.
    Sell_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask | ((Profit_Sell >= Profit_Threshold) | ((Profit_Buy <= Drop) & (Profit_Buy > 0))), axis = 0) + StartIndex + ReIndex

    # Calculate a mask - this keeps track of the earliest the next buy or sell is permitted.  For example if we
    # have a buy action then the next level will only allow actions on or AFTER this date (hence the double cumsum).
    # Buy_Rule[1] is the condition to buy to enter Level 1 (0.995)
    Buy_Mask = np.cumsum((Profit_Buy <= Buy_Config[1]) & (Profit_Buy>0) , axis = 0) > 0 
    
    # Store the results.
    Buy_Summary[0, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex + ReIndex
    
    # Now loop over the remaining levels ...
    for idx, x in enumerate(Buy_Config[2:]):

        # Compute the thresholds to buy or sell using the config data.
        Sell_Factor = Weighted_Config[idx+1] * Profit_Threshold
#         Buy_Factor  = Weighted_Config[idx+1] * x
        Buy_Factor  = x

        # Store the results (note: Buy_Mask keeps track of the last buy action)
        Sell_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Max_Mat_Mask |(((Profit_Sell>=Sell_Factor)| ((Profit_Buy <= Drop) & (Profit_Buy > 0))) & Buy_Mask), axis = 0) + StartIndex + ReIndex

        # Need to keep track of this for the next loop
        Buy_Mask = np.cumsum((Profit_Buy<=Buy_Factor) & Buy_Mask , axis = 0) > 0

        # Keep the results.
        Buy_Summary[idx+1, StartIndex:StartIndex + Adjust] = np.argmax(Buy_Mask, axis = 0) + StartIndex + ReIndex

    # Set up some arrays to store the results for the start dates.

    # First_Sell_Index: The index of the first sell action
    # Capital_Level   : The capital utilisation at the first sell action
    # Final_Price     : The closing price at the first sell action
    # Cost            : The weighted cost at the first sell action (this is level depended)
    First_Sell_Index, Capital_Level, Final_Price, Cost = np.empty(0), np.empty(0), np.empty(0), np.empty(0)

    # The date at the first sell action.
    End_Date = np.empty(0, dtype ='datetime64')

    # Find the first time the Sell date is lower than the buy date.  At this time the algo ends.
    # Need to determine what we do if we have equality ... which takes priority - here sell does.
    First_Sell_Level = np.argmax(np.cumsum(Buy_Summary >= Sell_Summary, axis=0) > 0, axis = 0) + 1

    # Calculate the index of the date for the first sell.
    for idx, i in enumerate(First_Sell_Level):
        First_Sell_Index = np.append(First_Sell_Index, Sell_Summary[i-1][idx])
        Capital_Level = np.append(Capital_Level, Capital_Config[i-1])
        Final_Price = np.append(Final_Price, One_High[int(Sell_Summary[i-1][idx])])
        Cost = np.append(Cost, Weighted_Config[i-1])
        End_Date = np.append(End_Date, One_Dates[int(Sell_Summary[i-1][idx])])

    # Collect the results in a dataframe    
    One_Results = pd.DataFrame({'Start_Date':One_Dates[0], 
                            'End_Date':End_Date,
                            'Initial_Open':One_Open[0],
                            'Final_Close':Final_Price,                        
                            'First_Sell':First_Sell_Index, 
                            'Level':First_Sell_Level-1,
                            'Capital' :Capital_Level,
                            'Cost':Cost})


    # Since we don't know the number of levels the level fields are calculated in a loop.
    for levels in range(Level_Length-1):
        One_Results['L'+str(levels+1)+'_Buy'] = Buy_Summary[int(levels)]
        One_Results['L'+str(levels+1)+'_Sell'] = Sell_Summary[int(levels)]

    One_Results['Life(s)'] = (One_Results['End_Date'] - One_Results['Start_Date']).astype('timedelta64[s]')

    # Calculate some Capital return.
    One_Results['Cap_Return%'] = One_Results['Capital'] * (One_Results['Final_Close'] / (One_Results['Initial_Open']*One_Results['Cost']) - 1)
    
    One_Results['Threshold'] = Profit_Threshold
    One_Results['Max_Drop'] = Drop
    One_Results['Max_Mat'] = Max_Mat
    
    One_Results['Sell_Price'] = round(One_Results['Initial_Open']*One_Results['Cost']*Profit_Threshold,2)
    
    
    return One_Results

print('* Run one High/Low function Constructed.')

print('')
Keep_Track()

* Run one High/Low function Constructed.

Executed successfully. Wed 25 May @ 12:51:05



3.3 Create some additional functions to metrics etc.

In [11]:
# For a selected start date show the dates and algo path (i.e. buy sell over time.)
# Interesting_Date: This is the date of interest.
# Results         : The output from the run functions above (e.g. Run())

def Algo_History(Interesting_Date, Results, Data):
    Sell = False

    # Find the data in the dataset or run it.
    Study = Results[Results['Start_Date']==Interesting_Date].copy()
    Study = Study.reset_index()
    print('* Start Date    :',Interesting_Date)
    print('* Opening Price :', Study['Initial_Open'][0])

    for i in range(30):
        if Sell == False:
            if Study['L'+str(i+1)+'_Buy'][0] < Study['L'+str(i+1)+'_Sell'][0]:
                print('* L{} Buy        :'.format(i+1), Data['start_timestamp'][int(Study['L'+str(i+1)+'_Buy'])])
            else:
                print('* L{} Sell       :'.format(i+1), Data['start_timestamp'][int(Study['L'+str(i+1)+'_Sell'])])
                Sell = True

print('* Algo track function defined.')    
print('')
Keep_Track()

* Algo track function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [12]:
# Same as Algo_History but now shows high/low and formats neater
# For a selected start date show the dates and algo path (i.e. buy sell over time.)
# Interesting_Date: This is the date of interest.
# Results         : The output from the run functions above (e.g. Run())

def Algo_History_Pretty(Interesting_Date, Results, Data):
    
    print('  Action    |          Date         |    Price    |    Type')
    print('------------+-----------------------+-------------+-----------')
#              Start       2022-04-01 00:08:00     3290.77    Opening    
    Sell = False


    # Find the data in the dataset or run it.
    Study = Results[Results['Start_Date']==Interesting_Date].copy()
    Study = Study.reset_index()
    print('  Start       ',Interesting_Date, '    ',Study['Initial_Open'][0], '      Open')

    for i in range(30):
        if Sell == False:
            if Study['L'+str(i+1)+'_Buy'][0] < Study['L'+str(i+1)+'_Sell'][0]:
                print('  L{} Buy      '.format(i+1), Data['start_timestamp'][int(Study['L'+str(i+1)+'_Buy'])],'    ',Data['low_price'][int(Study['L'+str(i+1)+'_Buy'])], '      Low')
            else:
                print('  L{} Sell     '.format(i+1), Data['start_timestamp'][int(Study['L'+str(i+1)+'_Sell'])],'    ',Data['high_price'][int(Study['L'+str(i+1)+'_Sell'])], '       High')
                Sell = True

    print('')
    print('* Life            :',np.array(Study['Life(s)'])[0],'seconds')
    print('* Final Sale price:', round(np.array(Study['Sell_Price'])[0],2))
    print('')
                
print('* Algo track function defined.')    
print('')
Keep_Track()

* Algo track function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [13]:
# For a selected start date calculate some metrics 
# Interesting_Date: This is the date of interest.
# Data            : The master dataset with all trading info.
# Results         : The output from the run functions above.
# Weighted_Config : Weighted cost from the config
# Capital_Config  : Capital allocation % from the config.

def Algo_Metrics(Interesting_Date, Data, Results, Weighted_Config, Capital_Config):
    
    Weights_Data = np.empty(0)
    Purchase_Data = np.empty(0)
    Capital_Data = np.empty(0)
    Sell = False

    # Find the data in the dataset
    Study = Results[Results['Start_Date']==Interesting_Date].copy()
    Study = Study.reset_index()  
    End_Date = Study['End_Date'][0]  
    
    # This the set of closing prices - the length should be correct.
    Closing_Data = np.array(Data[(Data['start_timestamp'] >= Interesting_Date) & (Data['start_timestamp'] <= End_Date)]['close_price'])
    Inital_Price = np.repeat(Study['Initial_Open'][0], len(Closing_Data))
    
    Start_Date = Interesting_Date
    Price_New =  End_Date = Study['Initial_Open'][0]  
    
    # Now we need to go ahead and populate the cost and capital sections given the buy/sell rules.
    for i in range(300):
        if Sell == False:
            if Study['L'+str(i+1)+'_Buy'][0] < Study['L'+str(i+1)+'_Sell'][0]:
                Date = Data['start_timestamp'][int(Study['L'+str(i+1)+'_Buy'])]
                
                Length = len(Data[(Data['start_timestamp'] >= Start_Date) & (Data['start_timestamp'] < Date)])
                
                # Create the Weights
                Weights_Data = np.append(Weights_Data, np.repeat(Weighted_Config[i], Length))
                
                # Create the new purchase price
                Purchase_Data = np.append(Purchase_Data, np.repeat(Price_New, Length))

                # Create the Capital allocation percentage
                Capital_Data = np.append(Capital_Data, np.repeat(Capital_Config[i], Length))
                
                # Prepare for the next loop
                Start_Date = Date
                Price_New = Data['close_price'][int(Study['L'+str(i+1)+'_Buy'])]
             
            else:
                Date = Data['start_timestamp'][int(Study['L'+str(i+1)+'_Sell'])]
                Length = len(Data[(Data['start_timestamp'] >= Start_Date) & (Data['start_timestamp'] <= Date)])
                
                # Create the Weights
                Weights_Data = np.append(Weights_Data, np.repeat(Weighted_Config[i], Length))
                Capital_Data = np.append(Capital_Data, np.repeat(Capital_Config[i], Length))
                Purchase_Data = np.append(Purchase_Data, np.repeat(Price_New, Length))
                
                Sell = True
    
    # Calculate the profit at each step.
    Profit_Data = Capital_Data * (Closing_Data / (Inital_Price*Weights_Data) - 1)/100
    return Inital_Price, Purchase_Data, Closing_Data, Weights_Data, Capital_Data, Profit_Data         

print('* Metric function defined.')    
print('')
Keep_Track()

* Metric function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [14]:
# This function loads the results of a prevous run.
def Load_Run(File_Name):
    print('* Loading', File_Name)
    
    try:
    
        # Load the transaction dataset.
        Data = pd.read_csv(File_Name)

        # Convert the time to a timestamp - need to specify the format is day first to avoid parsing errors.
        Data['start_timestamp'] = pd.to_datetime(Data['start_timestamp'], dayfirst=True)
        Data['End_Date'] = pd.to_datetime(Data['End_Date'], dayfirst=True)
        Data['Start_Date'] = pd.to_datetime(Data['Start_Date'], dayfirst=True)

        # Remove some values with end dates that are beyond the end or the data.
        print('* Filtering end dates beyond the end of the dataset.')
        Data = Data[Data['End_Date'] <= np.max(Data['start_timestamp'])]

        # Provide a quick summary of how many records we have
        print('* Data loaded   :', len(Data),'records.')
        print('* First Date    :', np.min(Data['start_timestamp']))
        print('* Last trade    :', np.max(Data['start_timestamp']))
        
        print('* ---------- Load complete.')
        
        return Data
    
    except:
        print('* ---------- Load FAILED!')

print('* Load result function defined.')    
print('')
Keep_Track()

* Load result function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [15]:
# These functions calculate the drawdown and Sharpe metrics for a given set of data
def Calculate_Drawdown(cum_rets):
    # Calculate the running maximum
    running_max = np.maximum.accumulate(cum_rets.dropna())
    
    # Ensure the value never drops below 1
    running_max[running_max < 1] = 1

    # Calculate the percentage drawdown - i.e. how far down am I on the running max at that point.
    drawdown = (cum_rets) / running_max - 1
    return drawdown

def Calculate_Sharpe(min_rets):
    
    # In some cases there may be records that generate zero standard deviation.
    # In which case we can't divide by it.
    Deviation =  np.std(min_rets)
    
    if Deviation == 0:
        return 0 # Is this the best choice?
    else:
        return np.mean(min_rets) / Deviation

print('* Metric function defined.')    
print('')
Keep_Track()

* Metric function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [16]:
# This function faciliates running the metrics over each record.
def Sharpe(Start_Date, Data, Results, Weighted_Cost, Capital):
    
    # Extract capital weighted profits.
    _, _, _, _, _,Profit_Example = Algo_Metrics(Start_Date, Data, Results, Weighted_Cost, Capital)

    # Convert back to percentage
    Profit_Example = Profit_Example + 1
    
    # Copy Kevin type calculation.
    Minute_ret = pd.Series(Profit_Example).pct_change()
    cum_ret = (1+Minute_ret).cumprod()

    # Tina appears to use returns across all in sample - can't do that here.
    sharpe = Calculate_Sharpe(Minute_ret)
    drawdown = Calculate_Drawdown(cum_ret)
    
    # The max drop down from a max at any point.
    max_dd = drawdown.min()*100
    
    return [sharpe, max_dd]

print('* Metric facilitation function defined.')    
print('')
Keep_Track()

* Metric facilitation function defined.

Executed successfully. Wed 25 May @ 12:51:05



In [17]:
# This function faciliates running the metrics over each record.
def Performance_Function(Start_Date, Data, Results, Weighted_Cost, Capital):
    
    # Extract capital weighted profits.
    _, _, _, _, _,Profit_Example = Algo_Metrics(Start_Date, Data, Results, Weighted_Cost, Capital)

    # Convert back to percentage
    Profit_Example = Profit_Example + 1
    
    # Copy Kevin type calculation.
    Minute_ret = pd.Series(Profit_Example).pct_change() 
    
    return np.array(Minute_ret)[1:]

print('* Algo performance array function defined.')    
print('')
Keep_Track()

* Algo performance array function defined.

Executed successfully. Wed 25 May @ 12:51:05



#### 4. Analysis

In [18]:
# Load the data
Data, LastTrade = Load_Data(FileName)

* Loading ../OneDrive_1_5-6-2022/ethbusd(2022.4.1_5.10)_1min.csv
* Filtering ...
* Data loaded   : 57590 records.
* First Date    : 2022-04-01 00:00:00
* Last trade    : 2022-05-10 23:59:00
* Last Sell Date: 2032-05-10 23:59:00 --added to ensure at least one sell date.
* Last Buy Date : 2052-05-10 23:59:00 --added to ensure at least one buy date.
* ---------- Load complete.


In [19]:
# Create the config
Buy_Rule, Weighted_Cost, Capital, Config = Load_Config(Load_file = True, File_Name = ConfigFile)

* Loading file: ../OneDrive_1_5-6-2022/029_2_005_006.csv
* Config loaded: 30 records.
* ---------- Load complete.


In [ ]:
# Run the model over all data (~ 30mins for 60k timestamps)
Out_All = Run_HL(Data, Buy_Rule, Weighted_Cost, Capital, Drop = 0.5, Max_Mat = 1200, Steps = 10 , Profit_Threshold = 1.006)
# Out_All = Run(Data, Buy_Rule, Weighted_Cost, Capital, Drop = 0.5, Max_Mat = 1200, Steps = 10 , Profit_Threshold = 1.006)

print('* Removing added dates')
Out_All = Out_All[(Out_All['Start_Date'] <= LastTrade) & (Out_All['End_Date'] <= LastTrade)]

print('* Joining high/low and trading data')
Out_All = Out_All.merge(Data, left_on = 'Start_Date', right_on = 'start_timestamp')

print('')
Keep_Track()

#### 4.1 Add in some Sharpe and Maxdraw down results for each run.

In [ ]:
# Run calculation over all records.
print('* Calculating Sharpe and Drawdown for each run ...')
Out_All['Metrics'] =  Out_All.apply(lambda row : Sharpe(row['Start_Date'], Data, Out_All, Weighted_Cost, Capital), axis = 1)
Out_All = Out_All.Metrics.apply(pd.Series).rename(columns = {0:'Sharpe',1:'Max_Drawdown'}).merge(Out_All, right_index = True, left_index = True)

print('')
Keep_Track()

#### 5. Save

In [ ]:
# SaveFile   = '../OneDrive_1_5-6-2022/ethbusd_1min_Results_Maturity.csv'
print('* Saving file to', SaveFile)
Out_All.to_csv(SaveFile, index=False)

print('')
Keep_Track()

#### Testing ...

In [20]:
Out_All = pd.read_csv('../OneDrive_1_5-6-2022/ethbusd_1min_Results.csv')

Run a single date/time

In [26]:
# Start_Date = '2022-04-01 14:10:00'
Start_Date = '2022-04-01 00:08:00'
Out = Run_One_Date_HL(Start_Date, 1.006, Data, Buy_Rule, Weighted_Cost, Capital, Drop = 0.5, Max_Mat = 1200)
Out[['Start_Date','End_Date','Life(s)','Initial_Open','Final_Close', 'Sell_Price','Level', 'Capital','Cap_Return%', 'Threshold','Max_Drop','Max_Mat']]

,Start_Date,End_Date,Life(s),Initial_Open,Final_Close,Sell_Price,Level,Capital,Cap_Return%,Threshold,Max_Drop,Max_Mat
0,2022-04-01 00:08:00,2022-04-01 07:29:00,26460.0,3290.77,3275.6,3274.56,4,8.06,0.050946,1.006,0.5,1200


In [23]:
# A few examples dates.
Start_Date = ['2022-04-01 00:01:00','2022-04-01 00:02:00','2022-04-01 00:03:00','2022-04-01 00:04:00','2022-04-01 00:05:00','2022-04-01 00:06:00','2022-04-01 00:07:00', '2022-04-01 00:08:00']
Out_All[Out_All['Start_Date'].isin(Start_Date)][['Start_Date','End_Date','Life(s)','Initial_Open','Final_Close','Sell_Price' ,'Level', 'Capital','Cap_Return%','Threshold','Max_Drop','Max_Mat']]

,Start_Date,End_Date,Life(s),Initial_Open,Final_Close,Sell_Price,Level,Capital,Cap_Return%,Threshold,Max_Drop,Max_Mat
1,2022-04-01 00:01:00,2022-04-01 00:41:00,2400.0,3280.41,3300.73,3300.09,0,1.39,0.008610,1.006,0.5,1200
2,2022-04-01 00:02:00,2022-04-01 00:12:00,600.0,3274.57,3295.94,3294.22,0,1.39,0.009071,1.006,0.5,1200
3,2022-04-01 00:03:00,2022-04-01 00:45:00,2520.0,3287.00,3306.80,3306.72,0,1.39,0.008373,1.006,0.5,1200
4,2022-04-01 00:04:00,2022-04-01 00:42:00,2280.0,3281.56,3303.43,3301.25,0,1.39,0.009264,1.006,0.5,1200
5,2022-04-01 00:05:00,2022-04-01 00:45:00,2400.0,3287.04,3306.80,3306.76,0,1.39,0.008356,1.006,0.5,1200
6,2022-04-01 00:06:00,2022-04-01 00:46:00,2400.0,3287.80,3307.91,3307.53,0,1.39,0.008502,1.006,0.5,1200
7,2022-04-01 00:07:00,2022-04-01 00:46:00,2340.0,3288.00,3307.91,3307.73,0,1.39,0.008417,1.006,0.5,1200
8,2022-04-01 00:08:00,2022-04-01 07:29:00,26460.0,3290.77,3275.60,3274.56,4,8.06,0.050946,1.006,0.5,1200


In [ ]:
# A few examples dates.
Start_Date = '2022-04-01 01:09:00'
Start_Date = '2022-04-01 00:08:00'
Out_All[Out_All['Start_Date']==Start_Date][['Start_Date','End_Date','Life(s)','Initial_Open','Final_Close', 'Sell_Price','Level', 'Capital','Cap_Return%','Threshold','Max_Drop','Max_Mat']]

#### History using full run

In [27]:
Algo_History_Pretty(Start_Date, Out_All, Data)

  Action    |          Date         |    Price    |    Type
------------+-----------------------+-------------+-----------
  Start        2022-04-01 00:08:00      3290.77       Open
  L1 Buy       2022-04-01 01:47:00      3274.05       Low
  L2 Buy       2022-04-01 01:54:00      3247.07       Low
  L3 Buy       2022-04-01 02:19:00      3237.71       Low
  L4 Buy       2022-04-01 02:26:00      3223.84       Low
  L5 Sell      2022-04-01 07:29:00      3275.6        High

* Life            : 26460.0 seconds
* Final Sale price: 3274.56



#### History using one date only - need to reset index to pick up records correctly

In [28]:
Algo_History_Pretty(Start_Date, Out, Data[Data['start_timestamp']>=Start_Date].reset_index())

  Action    |          Date         |    Price    |    Type
------------+-----------------------+-------------+-----------
  Start        2022-04-01 00:08:00      3290.77       Open
  L1 Buy       2022-04-01 01:47:00      3274.05       Low
  L2 Buy       2022-04-01 01:54:00      3247.07       Low
  L3 Buy       2022-04-01 02:19:00      3237.71       Low
  L4 Buy       2022-04-01 02:26:00      3223.84       Low
  L5 Sell      2022-04-01 07:29:00      3275.6        High

* Life            : 26460.0 seconds
* Final Sale price: 3274.56

